##**Ingeniería de Variables**

In [1]:
# --- Importaciones ---
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)

# Utilidad: reindexar por día dentro de cada estación para ventanas móviles estrictas
def reindex_daily_per_station(df, station_col='Estacion', date_col='Date'):
    out = []
    for est, dfg in df.groupby(station_col, sort=False):
        dfg = dfg.sort_values(date_col).copy()
        idx = pd.date_range(dfg[date_col].min(), dfg[date_col].max(), freq='D')
        dfg = dfg.set_index(date_col).reindex(idx).rename_axis(date_col).reset_index()
        dfg[station_col] = est
        out.append(dfg)
    return pd.concat(out, ignore_index=True)


In [9]:
url = "https://raw.githubusercontent.com/Sxmuu/TG-Samuel-P/main/Databases/Contam/Final/df_final12.xlsx"

df = pd.read_excel(url, engine="openpyxl")  # instala openpyxl si hace falta

In [10]:
expected_cols = ['Date','Estacion','Localidad','PM25','lat','lon','Altitud',
                 'Pres','Precip','Hum','Temp','WindSpeed']
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas esperadas: {missing}")

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values(['Estacion','Date']).drop_duplicates(subset=['Estacion','Date']).reset_index(drop=True)

print(df[['Estacion','Date']].groupby('Estacion').agg(['min','max','nunique']).head())


                                 Date                   
                                  min        max nunique
Estacion                                                
Centro De Alto Rendimiento 2021-01-01 2024-12-30    1460
Ciudad Bolivar             2021-01-01 2024-12-30    1460
Colina                     2021-01-15 2024-12-30    1446
Fontibon                   2021-01-01 2024-12-30    1460
Guaymaral                  2021-01-01 2024-12-30    1460


In [ ]:
# --- 2) (Opcional) Reindexar a diario por estación ---
# Si ya sabes que todas las estaciones tienen una observación por día, puedes saltarte esto.
# Si no, esto asegura ventanas móviles de longitud exacta (introducirá NaN si faltaban días).
df = reindex_daily_per_station(df, station_col='Estacion', date_col='Date')


In [11]:
# --- 2) Tipos y orden temporal ---
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# (Opcional) Orden global
df = df.sort_values(['Estacion', 'Date']).reset_index(drop=True)

# (Opcional) Quitar duplicados exactos por Estacion-Fecha (si existieran)
df = df.drop_duplicates(subset=['Estacion','Date'])
print(df[['Estacion','Date']].groupby('Estacion').agg(['min','max','nunique']).head())

                                 Date                   
                                  min        max nunique
Estacion                                                
Centro De Alto Rendimiento 2021-01-01 2024-12-30    1460
Ciudad Bolivar             2021-01-01 2024-12-30    1460
Colina                     2021-01-15 2024-12-30    1446
Fontibon                   2021-01-01 2024-12-30    1460
Guaymaral                  2021-01-01 2024-12-30    1460


In [12]:
# --- 3) Calendario y estacionalidad ---
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['dayofyear'] = df['Date'].dt.dayofyear
df['dow'] = df['Date'].dt.dayofweek
df['is_weekend'] = (df['dow'] >= 5).astype(int)
df['sin_doy'] = np.sin(2*np.pi*df['dayofyear']/365.25)
df['cos_doy'] = np.cos(2*np.pi*df['dayofyear']/365.25)

In [13]:
# --- 4) Funciones de lags y rolling (sin fuga) ---
def add_lags(df, group_key, date_col, vars_to_lag, lags):
    df = df.sort_values([group_key, date_col]).copy()
    for var in vars_to_lag:
        for k in lags:
            df[f'{var}_lag{k}'] = df.groupby(group_key, sort=False)[var].shift(k)
    return df

def add_rolling_features(df, group_key, date_col, var, windows, stats=('mean',), shift_one=True):
    df = df.sort_values([group_key, date_col]).copy()
    base = df.groupby(group_key, sort=False)[var]
    series = base.shift(1) if shift_one else base.transform(lambda x: x)
    for w in windows:
        roll = series.rolling(w)
        if 'mean' in stats:
            df[f'{var}_rollmean{w}'] = roll.mean().reset_index(level=0, drop=True)
        if 'std' in stats:
            df[f'{var}_rollstd{w}'] = roll.std().reset_index(level=0, drop=True)
        if 'min' in stats:
            df[f'{var}_rollmin{w}'] = roll.min().reset_index(level=0, drop=True)
        if 'max' in stats:
            df[f'{var}_rollmax{w}'] = roll.max().reset_index(level=0, drop=True)
    return df


In [14]:
# --- 5) Lags y rolling de PM25 (clave para 2026) ---
lags_pm25 = [1, 3, 7]
wins_pm25 = [3, 7]

df_feat = add_lags(df, group_key='Estacion', date_col='Date',
                   vars_to_lag=['PM25'], lags=lags_pm25)

df_feat = add_rolling_features(df_feat, group_key='Estacion', date_col='Date',
                               var='PM25', windows=wins_pm25,
                               stats=('mean',), shift_one=True)


In [15]:
# --- 6) (Opcional) Lags de meteorología (sí disponibles si tienes meteo 2026 o usarás climatología)
include_meteo_lags = True
meteo_vars = ['Temp','Hum','WindSpeed','Precip','Pres']
if include_meteo_lags:
    df_feat = add_lags(df_feat, group_key='Estacion', date_col='Date',
                       vars_to_lag=meteo_vars, lags=[1, 3])


In [16]:
# --- 8) Limpieza por NaN de bordes (debidos a lags/rolling) ---
rows_before = len(df_feat)
df_model = df_feat.dropna(subset=['PM25_lag1','PM25_rollmean3']).reset_index(drop=True)
rows_after = len(df_model)
print(f"Filas antes: {rows_before:,} | después de dropna: {rows_after:,} | perdidas: {rows_before - rows_after:,}")


Filas antes: 21,803 | después de dropna: 21,758 | perdidas: 45


In [17]:
# --- 9) Columnas finales para modelado (no entrenamos aún) ---
base_cols = ['Date','Estacion','Localidad','lat','lon','Altitud','PM25',
             'year','month','dayofyear','dow','is_weekend','sin_doy','cos_doy']
lag_cols = [c for c in df_model.columns if c.startswith('PM25_lag') or c.startswith('PM25_rollmean')]
met_lag_cols = [c for c in df_model.columns if any(c.startswith(v+'_lag') for v in meteo_vars)]

cols_for_next_steps = base_cols + lag_cols + met_lag_cols
df_ready = df_model[cols_for_next_steps].copy()

print("Columnas finales (primeras 25):")
print(df_ready.columns.tolist()[:25], '...')
df_ready.head()


Columnas finales (primeras 25):
['Date', 'Estacion', 'Localidad', 'lat', 'lon', 'Altitud', 'PM25', 'year', 'month', 'dayofyear', 'dow', 'is_weekend', 'sin_doy', 'cos_doy', 'PM25_lag1', 'PM25_lag3', 'PM25_lag7', 'PM25_rollmean3', 'PM25_rollmean7', 'Temp_lag1', 'Temp_lag3', 'Hum_lag1', 'Hum_lag3', 'WindSpeed_lag1', 'WindSpeed_lag3'] ...


,Date,Estacion,Localidad,lat,lon,Altitud,PM25,year,month,dayofyear,dow,is_weekend,sin_doy,cos_doy,PM25_lag1,PM25_lag3,PM25_lag7,PM25_rollmean3,PM25_rollmean7,Temp_lag1,Temp_lag3,Hum_lag1,Hum_lag3,WindSpeed_lag1,WindSpeed_lag3,Precip_lag1,Precip_lag3,Pres_lag1,Pres_lag3
0,2021-01-04,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,8.541667,2021,1,4,0,0,0.068755,0.997634,6.625000,15.763900,NaN,9.629633,NaN,18.16,19.17,84.79,79.44,0.77,0.83,11.61,4.03,82.26,82.30
1,2021-01-05,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,13.739130,2021,1,5,1,0,0.085906,0.996303,8.541667,6.500000,NaN,7.222222,NaN,18.37,19.64,81.29,77.80,1.17,0.75,12.97,3.90,82.18,82.28
2,2021-01-06,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,15.208333,2021,1,6,2,0,0.103031,0.994678,13.739130,6.625000,NaN,9.635266,NaN,17.87,18.16,78.17,84.79,0.99,0.77,2.28,11.61,82.20,82.26
3,2021-01-07,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,8.583333,2021,1,7,3,0,0.120126,0.992759,15.208333,8.541667,NaN,12.496377,NaN,17.92,18.37,77.14,81.29,0.68,1.17,1.89,12.97,82.30,82.18
4,2021-01-08,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,12.437500,2021,1,8,4,0,0.137185,0.990545,8.583333,13.739130,15.7639,12.510266,10.708766,18.81,17.87,75.35,78.17,0.66,0.99,3.63,2.28,82.34,82.20


In [ ]:
# --- 10) Guardar dataset de características ---
df_ready.to_csv('df_features_PM25_no_copollutants.csv', index=False)
print("✅ Guardado: df_features_PM25_no_copollutants.csv")


✅ Guardado: df_features_PM25_no_copollutants.csv


In [19]:
# --- Climatología meteo por estación y día-del-año (mediana) ---
years_hist = [2021, 2022, 2023, 2024]   # ajusta si procede
meteo_vars = ['Temp','Hum','WindSpeed','Precip','Pres']

df_hist = df[df['year'].isin(years_hist)].copy()
df_hist['doy'] = df_hist['Date'].dt.dayofyear

clima = (df_hist.groupby(['Estacion','doy'])[meteo_vars]
         .median()
         .reset_index()
         .rename(columns={v: f'{v}_clim' for v in meteo_vars}))

# Construir calendario 2026 y “pegar” climatología por estación y DOY
cal2026 = pd.date_range('2026-01-01','2026-12-31',freq='D')
cal = (pd.DataFrame({'Date': cal2026})
       .assign(doy=lambda x: x['Date'].dt.dayofyear)
      )

# Ejemplo: climatología para todas las estaciones (repetimos por estación)
ests = df['Estacion'].dropna().unique()
clima2026 = (cal.assign(key=1)
               .merge(pd.DataFrame({'Estacion': ests, 'key':1}), on='key')
               .drop(columns='key')
               .merge(clima, on=['Estacion','doy'], how='left'))

clima2026.to_csv('climatologia_meteo_2026_por_estacion.csv', index=False)
print("✅ Guardado: climatologia_meteo_2026_por_estacion.csv (medianas por DOY y estación)")


✅ Guardado: climatologia_meteo_2026_por_estacion.csv (medianas por DOY y estación)


##**Validación Cruzada**

In [20]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error



df = df_ready.copy()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Chequeo rápido:
print(df.shape)
print(df[['Date','Estacion','Localidad']].head(3))


(21758, 29)
        Date                    Estacion       Localidad
0 2021-01-04  Centro De Alto Rendimiento  Barrios Unidos
1 2021-01-05  Centro De Alto Rendimiento  Barrios Unidos
2 2021-01-06  Centro De Alto Rendimiento  Barrios Unidos


In [21]:
# Columnas base (ajusta si cambiaste nombres)
base_cols = ['Date','Localidad','lat','lon','Altitud','PM25',
             'year','month','dayofyear','dow','is_weekend','sin_doy','cos_doy']

# Lags/rollings ya construidos en Paso 1
lag_cols = [c for c in df.columns if c.startswith('PM25_lag') or c.startswith('PM25_rollmean')]
# Lags meteo si los añadiste en Paso 1
met_vars = ['Temp','Hum','WindSpeed','Precip','Pres']
met_lag_cols = [c for c in df.columns if any(c.startswith(v+'_lag') for v in met_vars)]

# Sin copolutantes:
feature_cols = ['Localidad','lat','lon','Altitud',
                'year','month','dayofyear','dow','is_weekend','sin_doy','cos_doy'] \
               + lag_cols + met_lag_cols

# Quitar filas con NaN en features/target (bordes por lags)
data = df.dropna(subset=feature_cols + ['PM25']).copy()

X = data[feature_cols].copy()
y = data['PM25'].values
dates = data['Date'].copy()
localities = data['Localidad'].copy()

# Preprocesamiento: OHE para categóricas; numéricas 'passthrough'
cat_features = ['Localidad']
num_features = [c for c in feature_cols if c not in cat_features]

pre = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
    ('num', 'passthrough', num_features)
])


In [22]:
model = HistGradientBoostingRegressor(
    learning_rate=0.06,
    max_iter=400,
    min_samples_leaf=25,
    early_stopping=True,
    random_state=42
)

pipe = Pipeline([
    ('pre', pre),
    ('model', model)
])


In [23]:
def build_time_folds(unique_dates, n_folds=4):
    """
    Forward-chaining con validación por bloques de igual tamaño aproximado.
    Devuelve lista de dicts con índices booleanos para train/val.
    """
    unique_dates = np.array(sorted(unique_dates))
    folds = []
    val_block = int(len(unique_dates)/(n_folds+1))
    for k in range(1, n_folds+1):
        train_end = k*val_block
        val_start = train_end
        val_end = val_start + val_block
        train_dates = unique_dates[:train_end]
        val_dates = unique_dates[val_start:val_end]
        folds.append({
            "train_dates": train_dates,
            "val_dates": val_dates
        })
    return folds

unique_days = np.array(sorted(dates.dt.normalize().unique()))
folds = build_time_folds(unique_days, n_folds=4)
[(f["train_dates"].min(), f["train_dates"].max(), f["val_dates"].min(), f["val_dates"].max()) for f in folds]


[(Timestamp('2021-01-08 00:00:00'),
  Timestamp('2021-10-24 00:00:00'),
  Timestamp('2021-10-25 00:00:00'),
  Timestamp('2022-08-10 00:00:00')),
 (Timestamp('2021-01-08 00:00:00'),
  Timestamp('2022-08-10 00:00:00'),
  Timestamp('2022-08-11 00:00:00'),
  Timestamp('2023-05-27 00:00:00')),
 (Timestamp('2021-01-08 00:00:00'),
  Timestamp('2023-05-27 00:00:00'),
  Timestamp('2023-05-28 00:00:00'),
  Timestamp('2024-03-12 00:00:00')),
 (Timestamp('2021-01-08 00:00:00'),
  Timestamp('2024-03-12 00:00:00'),
  Timestamp('2024-03-13 00:00:00'),
  Timestamp('2024-12-27 00:00:00'))]

In [24]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn
print("scikit-learn versión:", sklearn.__version__)

def rmse_compat(y_true, y_pred):
    """RMSE compatible con cualquier versión de scikit-learn."""
    try:
        # sklearn >= 0.22 (aprox.) soporta 'squared'
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        # fallback para versiones antiguas
        return np.sqrt(mean_squared_error(y_true, y_pred))


scikit-learn versión: 1.6.1


In [25]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def rmse_compat(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

# Cambiar agrupamiento de 'Estacion' a 'Localidad'
def eval_by_locality(y_true, y_pred, localities):
    dfm = pd.DataFrame({"y": y_true, "yhat": y_pred, "Localidad": localities})  # Cambio aquí
    out = []
    for loc, g in dfm.groupby("Localidad", sort=False):  # Cambiar 'Estacion' por 'Localidad'
        mae = mean_absolute_error(g["y"], g["yhat"])
        rmse = rmse_compat(g["y"], g["yhat"])   # Calculamos RMSE
        out.append({"Localidad": loc, "MAE": mae, "RMSE": rmse, "n": len(g)})  # Cambiar 'Estacion' por 'Localidad'
    return pd.DataFrame(out).sort_values("RMSE")

# Cambiar 'stations' a 'localities'
cv_rows = []
per_locality_reports = []

for i, f in enumerate(folds, start=1):
    tr_mask = dates.dt.normalize().isin(f["train_dates"])
    va_mask = dates.dt.normalize().isin(f["val_dates"])

    X_tr, y_tr = X[tr_mask], y[tr_mask]
    X_va, y_va = X[va_mask], y[va_mask]
    locality_va = localities[va_mask].values  # Usamos 'localities' aquí en lugar de 'stations'

    pipe.fit(X_tr, y_tr)
    pred_va = pipe.predict(X_va)

    mae = mean_absolute_error(y_va, pred_va)
    rmse = rmse_compat(y_va, pred_va)  # Calculamos RMSE

    cv_rows.append({
        "fold": i,
        "train_start": str(dates[tr_mask].min().date()),
        "train_end":   str(dates[tr_mask].max().date()),
        "val_start":   str(dates[va_mask].min().date()),
        "val_end":     str(dates[va_mask].max().date()),
        "n_train": int(tr_mask.sum()),
        "n_val": int(va_mask.sum()),
        "MAE": mae,
        "RMSE": rmse
    })

    # Usamos la nueva función con 'Localidad'
    rep = eval_by_locality(y_va, pred_va, locality_va)
    rep.insert(0, 'fold', i)
    per_locality_reports.append(rep)

cv_table = pd.DataFrame(cv_rows)
per_locality_table = pd.concat(per_locality_reports, ignore_index=True)

display(cv_table.round(3))
display(per_locality_table.round(3))

# Guardamos los resultados
cv_table.to_csv("cv_temporal_global.csv", index=False)
per_locality_table.to_csv("cv_temporal_por_localidad.csv", index=False)
print("✅ Guardados: cv_temporal_global.csv, cv_temporal_por_localidad.csv")


,fold,train_start,train_end,val_start,val_end,n_train,n_val,MAE,RMSE
0,1,2021-01-08,2021-10-24,2021-10-25,2022-08-10,4253,4350,4.237,5.534
1,2,2021-01-08,2022-08-10,2022-08-11,2023-05-27,8603,4350,4.537,5.920
2,3,2021-01-08,2023-05-27,2023-05-28,2024-03-12,12953,4350,4.251,5.584
3,4,2021-01-08,2024-03-12,2024-03-13,2024-12-27,17303,4350,3.976,5.343


,fold,Localidad,MAE,RMSE,n
0,1,Suba,3.421,4.435,870
1,1,Engativa,3.872,4.975,290
2,1,Barrios Unidos,3.973,5.015,290
3,1,Kennedy,4.260,5.388,290
4,1,Santa Fe,4.218,5.424,290
5,1,San Cristobal,4.033,5.454,290
6,1,Puente Aranda,4.378,5.469,580
7,1,Fontibon,4.600,5.890,580
8,1,Usme,4.567,6.218,290
9,1,Tunjuelito,4.948,6.517,290


✅ Guardados: cv_temporal_global.csv, cv_temporal_por_localidad.csv


In [26]:
mask_train = dates.dt.year <= 2023
mask_test  = dates.dt.year == 2024

X_tr, y_tr = X[mask_train], y[mask_train]
X_te, y_te = X[mask_test],  y[mask_test]
localities_te = localities[mask_test].values  # Cambiar 'stations' por 'localities'

pipe.fit(X_tr, y_tr)
pred_te = pipe.predict(X_te)

mae_te = mean_absolute_error(y_te, pred_te)
rmse_te = rmse_compat(y_te, pred_te)  # Calculando RMSE sin squared=False
print(f"Hold-out 2024 → MAE: {mae_te:.3f} | RMSE: {rmse_te:.3f} | n_test: {mask_test.sum()}")

# Evaluar por localidad en vez de estación
rep_te = eval_by_locality(y_te, pred_te, localities_te).round(3)  # Usar eval_by_locality
display(rep_te)

# Guardar el resultado con nombre modificado
rep_te.to_csv("holdout2024_por_localidad.csv", index=False)
print("✅ Guardado: holdout2024_por_localidad.csv")

Hold-out 2024 → MAE: 4.340 | RMSE: 5.572 | n_test: 5475


,Localidad,MAE,RMSE,n
2,Suba,3.374,4.448,1095
8,San Cristobal,3.833,4.989,365
9,Tunjuelito,4.015,5.038,365
10,Usme,4.024,5.314,365
0,Barrios Unidos,4.217,5.465,365
6,Engativa,4.189,5.489,365
7,Santa Fe,4.333,5.623,365
5,Kennedy,4.470,5.689,365
3,Fontibon,4.782,6.098,730
4,Puente Aranda,5.597,6.482,730


✅ Guardado: holdout2024_por_localidad.csv


In [27]:
# Repite el armado del fold 4 exactamente como en tu CV
fold = folds[3]  # 4º fold (índice 3)
tr_mask = dates.dt.normalize().isin(fold["train_dates"])
va_mask = dates.dt.normalize().isin(fold["val_dates"])

pipe.fit(X[tr_mask], y[tr_mask])
pred_va = pipe.predict(X[va_mask])

df_va = data.loc[va_mask, ['Date','Localidad','PM25']].copy()  # Cambiar 'Estacion' por 'Localidad'
df_va['yhat'] = pred_va
df_va['abs_err'] = (df_va['PM25'] - df_va['yhat']).abs()

# Modificar la función para agrupar por 'Localidad' en vez de 'Estacion'
def mae_rmse(g):
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np
    try:
        rmse = mean_squared_error(g['PM25'], g['yhat'], squared=False)
    except TypeError:
        rmse = np.sqrt(mean_squared_error(g['PM25'], g['yhat']))
    return pd.Series({
        'n': len(g),
        'MAE': mean_absolute_error(g['PM25'], g['yhat']),
        'RMSE': rmse
    })

# Agrupar por 'Localidad' en vez de 'Estacion'
print(df_va.groupby('Localidad').apply(mae_rmse).round(3))

# Si quieres ver la distribución de errores de Usaquen (ahora por localidad)
usa = df_va[df_va['Localidad'] == 'Puente Aranda']  # Cambiar 'Estacion' por 'Localidad'
print(usa[['abs_err']].describe(percentiles=[.5,.9,.95,.99]).round(3).T)
print("Fechas con mayor error en Puente Aranda:\n", usa.nlargest(5, 'abs_err')[['Date', 'PM25', 'yhat', 'abs_err']])


                    n    MAE   RMSE
Localidad                          
Barrios Unidos  290.0  4.148  5.405
Ciudad Bolivar  290.0  4.977  6.599
Engativa        290.0  4.045  5.493
Fontibon        580.0  4.698  6.119
Kennedy         290.0  4.439  5.710
Puente Aranda   580.0  3.449  5.062
San Cristobal   290.0  3.717  4.872
Santa Fe        290.0  4.260  5.610
Suba            870.0  3.276  4.375
Tunjuelito      290.0  3.892  4.943
Usme            290.0  4.036  5.437
         count   mean    std    min    50%    90%     95%     99%     max
abs_err  580.0  3.449  3.708  0.002  2.204  8.133  11.241  16.103  34.026
Fechas con mayor error en Puente Aranda:
             Date       PM25       yhat    abs_err
15656 2024-04-01  34.708504   0.682310  34.026194
8445  2024-03-23  48.570833  27.621081  20.949752
15658 2024-04-03   0.041667  19.795842  19.754175
8444  2024-03-22  48.000000  30.101864  17.898136
8471  2024-04-18  44.983333  27.633343  17.349990


/tmp/ipython-input-2936231327.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(df_va.groupby('Localidad').apply(mae_rmse).round(3))


##**CatBoost**

In [29]:
# ==== 0) Setup: instalar CatBoost (si hiciera falta) ====
!pip -q install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 26.5 MB/s eta 0:00:00


In [30]:
# ==== 1) Cargar librerías y datos ====
import json, math, random
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool

# Compatibilidad de métricas (RMSE con y sin 'squared')
from sklearn.metrics import mean_absolute_error, mean_squared_error
def rmse_compat(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

df = df_ready.copy()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Solo por seguridad: tipar categóricas como string
for c in ['Localidad']:  # Cambiar 'Estacion' por 'Localidad'
    if c in df.columns:
        df[c] = df[c].astype(str)

print(df.shape)
df.head(2)


(21758, 29)


,Date,Estacion,Localidad,lat,lon,Altitud,PM25,year,month,dayofyear,dow,is_weekend,sin_doy,cos_doy,PM25_lag1,PM25_lag3,PM25_lag7,PM25_rollmean3,PM25_rollmean7,Temp_lag1,Temp_lag3,Hum_lag1,Hum_lag3,WindSpeed_lag1,WindSpeed_lag3,Precip_lag1,Precip_lag3,Pres_lag1,Pres_lag3
0,2021-01-04,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,8.541667,2021,1,4,0,0,0.068755,0.997634,6.625000,15.7639,NaN,9.629633,NaN,18.16,19.17,84.79,79.44,0.77,0.83,11.61,4.03,82.26,82.30
1,2021-01-05,Centro De Alto Rendimiento,Barrios Unidos,4.65847,-74.08396,2552,13.739130,2021,1,5,1,0,0.085906,0.996303,8.541667,6.5000,NaN,7.222222,NaN,18.37,19.64,81.29,77.80,1.17,0.75,12.97,3.90,82.18,82.28


In [31]:
# ==== 2) Definir features y target (sin copolutantes) ====
base_cols = ['Date','Localidad','lat','lon','Altitud',  # Cambiar 'Estacion' por 'Localidad'
             'year','month','dayofyear','dow','is_weekend','sin_doy','cos_doy']

lag_cols = [c for c in df.columns if c.startswith('PM25_lag') or c.startswith('PM25_rollmean')]

met_vars = ['Temp','Hum','WindSpeed','Precip','Rad']
met_lag_cols = [c for c in df.columns if any(c.startswith(v+'_lag') for v in met_vars)]

# target
target_col = 'PM25'

# columnas finales de X (Date no va al modelo)
feature_cols = ['Localidad','lat','lon','Altitud',  # Cambiar 'Estacion' por 'Localidad'
                'year','month','dayofyear','dow','is_weekend','sin_doy','cos_doy'] \
               + lag_cols + met_lag_cols

data = df.dropna(subset=feature_cols + [target_col]).copy()
X = data[feature_cols].copy()
y = data[target_col].values
dates = data['Date'].copy()
localities = data['Localidad'].copy()  # Cambiar 'Estacion' por 'Localidad'

# Índices de categóricas para CatBoost (dentro de X)
cat_cols = ['Localidad']  # Cambiar 'Estacion' por 'Localidad'
cat_idx = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]

len(feature_cols), feature_cols[:8], cat_idx


(24,
 ['Localidad', 'lat', 'lon', 'Altitud', 'year', 'month', 'dayofyear', 'dow'],
 [0])

In [32]:
# ==== 3) Construir los mismos folds temporales (ventana expansiva) ====
def build_time_folds(unique_dates, n_folds=4):
    unique_dates = np.array(sorted(unique_dates))
    folds = []
    val_block = int(len(unique_dates)/(n_folds+1))
    for k in range(1, n_folds+1):
        train_end = k*val_block
        val_start = train_end
        val_end = val_start + val_block
        train_dates = unique_dates[:train_end]
        val_dates = unique_dates[val_start:val_end]
        folds.append({"train_dates": train_dates, "val_dates": val_dates})
    return folds

unique_days = np.array(sorted(dates.dt.normalize().unique()))
folds = build_time_folds(unique_days, n_folds=4)

[(f["train_dates"].min(), f["train_dates"].max(), f["val_dates"].min(), f["val_dates"].max()) for f in folds]


[(Timestamp('2021-01-08 00:00:00'),
  Timestamp('2021-10-24 00:00:00'),
  Timestamp('2021-10-25 00:00:00'),
  Timestamp('2022-08-10 00:00:00')),
 (Timestamp('2021-01-08 00:00:00'),
  Timestamp('2022-08-10 00:00:00'),
  Timestamp('2022-08-11 00:00:00'),
  Timestamp('2023-05-27 00:00:00')),
 (Timestamp('2021-01-08 00:00:00'),
  Timestamp('2023-05-27 00:00:00'),
  Timestamp('2023-05-28 00:00:00'),
  Timestamp('2024-03-12 00:00:00')),
 (Timestamp('2021-01-08 00:00:00'),
  Timestamp('2024-03-12 00:00:00'),
  Timestamp('2024-03-13 00:00:00'),
  Timestamp('2024-12-27 00:00:00'))]

In [33]:
# ==== 4) Función de evaluación para un set de hiperparámetros ====
def eval_params(params, verbose=False):
    """Devuelve dict con MAE y RMSE promediados en CV, y por fold."""
    fold_results = []
    for i, f in enumerate(folds, start=1):
        tr_mask = dates.dt.normalize().isin(f["train_dates"])
        va_mask = dates.dt.normalize().isin(f["val_dates"])

        X_tr, y_tr = X[tr_mask], y[tr_mask]
        X_va, y_va = X[va_mask], y[va_mask]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_idx)

        model = CatBoostRegressor(
            loss_function='RMSE',
            iterations=params.get('iterations', 2000),
            depth=params.get('depth', 7),
            learning_rate=params.get('learning_rate', 0.06),
            l2_leaf_reg=params.get('l2_leaf_reg', 3.0),
            bootstrap_type=params.get('bootstrap_type', 'Bayesian'),
            bagging_temperature=params.get('bagging_temperature', 1.0),
            random_strength=params.get('random_strength', 0.0),
            early_stopping_rounds=params.get('early_stopping_rounds', 100),
            random_seed=42,
            verbose=False,
            allow_writing_files=False
        )

        model.fit(train_pool, eval_set=valid_pool, use_best_model=True, verbose=verbose)
        pred_va = model.predict(valid_pool)

        mae = mean_absolute_error(y_va, pred_va)
        rmse = rmse_compat(y_va, pred_va)

        fold_results.append({"fold": i, "MAE": mae, "RMSE": rmse, "n_val": int(va_mask.sum())})

    mae_mean = float(np.mean([r["MAE"] for r in fold_results]))
    rmse_mean = float(np.mean([r["RMSE"] for r in fold_results]))
    return {"mae_mean": mae_mean, "rmse_mean": rmse_mean, "folds": fold_results}


In [35]:
# ==== 5) Random Search liviano de hiperparámetros ====
random.seed(42)
search_space = {
    "depth":       [5,6,7,8,9,10],
    "learning_rate": [0.02, 0.03, 0.04, 0.06, 0.08, 0.10],
    "l2_leaf_reg":  [1.0, 2.0, 3.0, 5.0, 7.0, 10.0],
    "bagging_temperature": [0.0, 0.5, 1.0, 2.0, 3.0, 5.0],
    "random_strength": [0.0, 0.1, 0.2, 0.5],
    "iterations":  [1500, 2000, 2500],
    "bootstrap_type": ['Bayesian'],
    "early_stopping_rounds": [100]
}

def sample_params(space):
    return {
        "depth": random.choice(space["depth"]),
        "learning_rate": random.choice(space["learning_rate"]),
        "l2_leaf_reg": random.choice(space["l2_leaf_reg"]),
        "bagging_temperature": random.choice(space["bagging_temperature"]),
        "random_strength": random.choice(space["random_strength"]),
        "iterations": random.choice(space["iterations"]),
        "bootstrap_type": 'Bayesian',
        "early_stopping_rounds": 100
    }

results = []
N_TRIALS = 24  # puedes subirlo si tienes tiempo de cómputo
for t in range(1, N_TRIALS+1):
    params = sample_params(search_space)
    res = eval_params(params, verbose=False)
    res["params"] = params
    results.append(res)
    print(f"Trial {t}/{N_TRIALS} → RMSE_CV={res['rmse_mean']:.3f} | MAE_CV={res['mae_mean']:.3f} | {params}")

# Ordenar por RMSE (menor es mejor)
results_sorted = sorted(results, key=lambda r: r["rmse_mean"])
best = results_sorted[0]
print("\n=== MEJOR CONFIGURACIÓN (CV) ===")
print(json.dumps(best["params"], indent=2))
print("CV → RMSE promedio:", round(best["rmse_mean"],3), " | MAE promedio:", round(best["mae_mean"],3))
pd.DataFrame(best["folds"]).round(3)


Trial 1/24 → RMSE_CV=5.667 | MAE_CV=4.347 | {'depth': 10, 'learning_rate': 0.02, 'l2_leaf_reg': 1.0, 'bagging_temperature': 5.0, 'random_strength': 0.2, 'iterations': 1500, 'bootstrap_type': 'Bayesian', 'early_stopping_rounds': 100}
Trial 2/24 → RMSE_CV=5.469 | MAE_CV=4.190 | {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 10.0, 'bagging_temperature': 0.0, 'random_strength': 0.0, 'iterations': 2500, 'bootstrap_type': 'Bayesian', 'early_stopping_rounds': 100}
Trial 3/24 → RMSE_CV=5.481 | MAE_CV=4.201 | {'depth': 8, 'learning_rate': 0.02, 'l2_leaf_reg': 1.0, 'bagging_temperature': 0.0, 'random_strength': 0.1, 'iterations': 1500, 'bootstrap_type': 'Bayesian', 'early_stopping_rounds': 100}
Trial 4/24 → RMSE_CV=5.567 | MAE_CV=4.269 | {'depth': 9, 'learning_rate': 0.08, 'l2_leaf_reg': 1.0, 'bagging_temperature': 3.0, 'random_strength': 0.1, 'iterations': 2500, 'bootstrap_type': 'Bayesian', 'early_stopping_rounds': 100}
Trial 5/24 → RMSE_CV=5.672 | MAE_CV=4.336 | {'depth': 10, 'learning_ra

,fold,MAE,RMSE,n_val
0,1,4.022,5.202,4350
1,2,4.445,5.778,4350
2,3,4.129,5.403,4350
3,4,3.926,5.130,4350


In [36]:
# ==== 6) Reentrenar con mejor set (2021–2023) y evaluar en 2024 (hold-out) ====
mask_train = (dates.dt.year <= 2023)
mask_test  = (dates.dt.year == 2024)

X_tr, y_tr = X[mask_train], y[mask_train]
X_te, y_te = X[mask_test],  y[mask_test]

train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
test_pool  = Pool(X_te, y_te, cat_features=cat_idx)

best_params = best["params"].copy()
final_model = CatBoostRegressor(
    loss_function='RMSE',
    iterations=best_params["iterations"],
    depth=best_params["depth"],
    learning_rate=best_params["learning_rate"],
    l2_leaf_reg=best_params["l2_leaf_reg"],
    bootstrap_type='Bayesian',
    bagging_temperature=best_params["bagging_temperature"],
    random_strength=best_params["random_strength"],
    early_stopping_rounds=100,
    random_seed=42,
    verbose=False,
    allow_writing_files=False
)

# Usamos un pequeño conjunto de validación (último mes de 2023) para early stopping del final_model
cutoff = pd.Timestamp('2023-12-01')
tr_in  = dates[mask_train] < cutoff
tr_val = (dates[mask_train] >= cutoff)

final_model.fit(
    Pool(X_tr[tr_in],  y_tr[tr_in],  cat_features=cat_idx),
    eval_set=Pool(X_tr[tr_val], y_tr[tr_val], cat_features=cat_idx),
    use_best_model=True, verbose=False
)

pred_te = final_model.predict(test_pool)
mae_te = mean_absolute_error(y_te, pred_te)
rmse_te = rmse_compat(y_te, pred_te)
print(f"Hold-out 2024 → MAE: {mae_te:.3f} | RMSE: {rmse_te:.3f} | n_test: {int(mask_test.sum())}")

# Guardar predicciones 2024
out_te = data.loc[mask_test, ['Date','Localidad',target_col]].copy()  # Cambiar 'Estacion' por 'Localidad'
out_te['yhat'] = pred_te
out_te.to_csv('predicciones_holdout2024_catboost.csv', index=False)

# Métricas por localidad (2024) - Cambiar 'Estacion' por 'Localidad'
by_locality = out_te.groupby('Localidad').apply(  # Cambiar 'Estacion' por 'Localidad'
    lambda g: pd.Series({
        'n': len(g),
        'MAE': mean_absolute_error(g[target_col], g['yhat']),
        'RMSE': rmse_compat(g[target_col], g['yhat'])
    })
).reset_index().sort_values('RMSE')
by_locality.round(3)


Hold-out 2024 → MAE: 4.362 | RMSE: 5.616 | n_test: 5475


/tmp/ipython-input-312011496.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_locality = out_te.groupby('Localidad').apply(  # Cambiar 'Estacion' por 'Localidad'


,Localidad,n,MAE,RMSE
9,Tunjuelito,365.0,3.577,4.446
8,Suba,1095.0,3.381,4.500
6,San Cristobal,365.0,3.749,4.811
10,Usme,365.0,4.011,5.216
0,Barrios Unidos,365.0,4.158,5.305
4,Kennedy,365.0,4.216,5.411
2,Engativa,365.0,4.067,5.459
7,Santa Fe,365.0,4.143,5.600
3,Fontibon,730.0,4.821,6.222
1,Ciudad Bolivar,365.0,5.150,6.623


In [37]:
# ==== 7) Importancias de variables y guardado del modelo ====
# Importancia "Feature Importance" de CatBoost (Gain)
fi = final_model.get_feature_importance(train_pool, type='FeatureImportance')
fi_df = pd.DataFrame({'feature': X.columns, 'importance': fi}).sort_values('importance', ascending=False)
fi_df.to_csv('feature_importance_catboost.csv', index=False)

# Guardar modelo y parámetros
final_model.save_model('catboost_pm25_model.cbm')
with open('best_params_catboost.json','w') as f:
    json.dump(best_params, f, indent=2)

print("✅ Guardados: catboost_pm25_model.cbm, best_params_catboost.json, feature_importance_catboost.csv, predicciones_holdout2024_catboost.csv")
fi_df.head(15)


✅ Guardados: catboost_pm25_model.cbm, best_params_catboost.json, feature_importance_catboost.csv, predicciones_holdout2024_catboost.csv


,feature,importance
11,PM25_lag1,36.092186
22,Precip_lag1,6.837925
7,dow,6.758154
10,cos_doy,6.535847
20,WindSpeed_lag1,4.717151
15,PM25_rollmean7,4.465127
2,lon,4.251887
6,dayofyear,3.656441
18,Hum_lag1,3.157391
9,sin_doy,2.868706


In [38]:
from sklearn.metrics import r2_score

# --- 1) Evaluación de varianza explicada (R²) para cada fold y hold-out 2024 ---
def evaluate_r2(y_true, y_pred):
    """Calcula R² (varianza explicada) global."""
    return r2_score(y_true, y_pred)

# --- 2) Evaluación en los folds de la CV temporal ---
def eval_r2_params(params, verbose=False):
    """Devuelve el R² promedio en CV y por fold."""
    fold_results = []
    for i, f in enumerate(folds, start=1):
        tr_mask = dates.dt.normalize().isin(f["train_dates"])
        va_mask = dates.dt.normalize().isin(f["val_dates"])

        X_tr, y_tr = X[tr_mask], y[tr_mask]
        X_va, y_va = X[va_mask], y[va_mask]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
        valid_pool = Pool(X_va, y_va, cat_features=cat_idx)

        model = CatBoostRegressor(
            loss_function='RMSE',
            iterations=params.get('iterations', 2000),
            depth=params.get('depth', 7),
            learning_rate=params.get('learning_rate', 0.06),
            l2_leaf_reg=params.get('l2_leaf_reg', 3.0),
            bootstrap_type='Bayesian',
            bagging_temperature=params.get('bagging_temperature', 1.0),
            random_strength=params.get('random_strength', 0.0),
            early_stopping_rounds=100,
            random_seed=42,
            verbose=False,
            allow_writing_files=False
        )
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True, verbose=verbose)

        # Predicciones para R² (validación)
        yhat_va = model.predict(valid_pool)

        # Calcular R² por fold
        r2 = evaluate_r2(y_va, yhat_va)

        fold_results.append({"fold": i, "R²": r2, "n_val": int(va_mask.sum())})

    r2_mean = float(np.mean([r["R²"] for r in fold_results]))
    return {"r2_mean": r2_mean, "folds": fold_results}

# Evaluación del mejor modelo (con parámetros encontrados en la búsqueda)
r2_result = eval_r2_params(best["params"])
print("CV (varianza explicada R²) → R² promedio:", round(r2_result["r2_mean"],3))
pd.DataFrame(r2_result["folds"]).round(3)


CV (varianza explicada R²) → R² promedio: 0.585


,fold,R²,n_val
0,1,0.532,4350
1,2,0.470,4350
2,3,0.633,4350
3,4,0.704,4350


In [39]:
# --- 3) Evaluación en hold-out 2024 (R²) ---
mask_train = (dates.dt.year <= 2023)
mask_test  = (dates.dt.year == 2024)

X_tr, y_tr = X[mask_train], y[mask_train]
X_te, y_te = X[mask_test],  y[mask_test]

train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
test_pool  = Pool(X_te, y_te, cat_features=cat_idx)

final_model = CatBoostRegressor(
    loss_function='RMSE',
    iterations=best["params"]["iterations"],
    depth=best["params"]["depth"],
    learning_rate=best["params"]["learning_rate"],
    l2_leaf_reg=best["params"]["l2_leaf_reg"],
    bootstrap_type='Bayesian',
    bagging_temperature=best["params"]["bagging_temperature"],
    random_strength=best["params"]["random_strength"],
    early_stopping_rounds=100,
    random_seed=42,
    verbose=False,
    allow_writing_files=False
)

final_model.fit(train_pool, eval_set=test_pool, use_best_model=True, verbose=False)

# Predicciones en hold-out
yhat_te = final_model.predict(test_pool)

# Calcular R² en hold-out
r2_te = evaluate_r2(y_te, yhat_te)
print(f"Hold-out 2024 → R²: {r2_te:.3f} | n_test: {int(mask_test.sum())}")


Hold-out 2024 → R²: 0.672 | n_test: 5475


In [40]:
# --- 4) R² por estación en hold-out 2024 ---
def eval_r2_by_station(y_true, y_pred, localities):
    """Calcula R² por estación."""
    dfm = pd.DataFrame({"y": y_true, "yhat": y_pred, "Localidad": localities})
    out = []
    for est, g in dfm.groupby("Localidad", sort=False):
        r2 = evaluate_r2(g["y"], g["yhat"])
        out.append({"Localidad": est, "R²": r2, "n": len(g)})
    return pd.DataFrame(out).sort_values("R²", ascending=False)

r2_by_station = eval_r2_by_station(y_te, yhat_te, localities[mask_test])
r2_by_station.round(3)


,Localidad,R²,n
2,Suba,0.721,1095
8,San Cristobal,0.682,365
6,Engativa,0.646,365
0,Barrios Unidos,0.644,365
10,Usme,0.633,365
7,Santa Fe,0.632,365
5,Kennedy,0.622,365
3,Fontibon,0.615,730
1,Ciudad Bolivar,0.573,365
4,Puente Aranda,0.536,730
